### 文本的预处理
关于这个模块，你并不需要完成任何任务，所有的模块已经写好，你只需要读一读就可以了。输入为question_answer.txt，最后处理之后的结果存放在question_answer_parse.pkl文件中。

In [2]:
import numpy as np
import pandas as pd
import emoji
import re
import jieba
import os
from collections import Counter

In [3]:
# 导入原始数据
QApares_df = pd.read_csv('data/question_answer.txt',sep='\t',header=None)

In [4]:
#观察原始数据格式，查看头几个样本
QApares_df.head()

,0,1
0,买 二份 有没有 少点 呀,亲亲 真的 不好意思 我们 已经 是 优惠价 了 呢 小本生意 请亲 谅解
1,那 就 等 你们 处理 喽,好 的 亲退 了
2,那 我 不 喜欢,颜色 的话 一般 茶刀 茶针 和 二合一 的话 都 是 红木 檀 和 黑木 檀 哦
3,不是 免 运费,本店 茶具 订单 满 99 包邮除 宁夏 青海 内蒙古 海南 新疆 西藏 满 39 包邮
4,好吃 吗,好吃 的


In [5]:
# 将 QApares_df列重命名
QApares_df.columns = ['question_before_preprocessing','answer']
QApares_df.head()

,question_before_preprocessing,answer
0,买 二份 有没有 少点 呀,亲亲 真的 不好意思 我们 已经 是 优惠价 了 呢 小本生意 请亲 谅解
1,那 就 等 你们 处理 喽,好 的 亲退 了
2,那 我 不 喜欢,颜色 的话 一般 茶刀 茶针 和 二合一 的话 都 是 红木 檀 和 黑木 檀 哦
3,不是 免 运费,本店 茶具 订单 满 99 包邮除 宁夏 青海 内蒙古 海南 新疆 西藏 满 39 包邮
4,好吃 吗,好吃 的


In [6]:
def rm_space(sentence):
    '''将已分好词的句子去掉词之间的空格变成原句子，如将‘那 就 等 你们 处理 喽’改为‘那就等你们处理喽’'''
    return ''.join(sentence.split())
QApares_df['question'] = QApares_df.question_before_preprocessing.apply(rm_space)
QApares_df['answer'] = QApares_df.answer.apply(rm_space)

In [7]:
QApares_df.head()

,question_before_preprocessing,answer,question
0,买 二份 有没有 少点 呀,亲亲真的不好意思我们已经是优惠价了呢小本生意请亲谅解,买二份有没有少点呀
1,那 就 等 你们 处理 喽,好的亲退了,那就等你们处理喽
2,那 我 不 喜欢,颜色的话一般茶刀茶针和二合一的话都是红木檀和黑木檀哦,那我不喜欢
3,不是 免 运费,本店茶具订单满99包邮除宁夏青海内蒙古海南新疆西藏满39包邮,不是免运费
4,好吃 吗,好吃的,好吃吗


接下来对数据进行预处理

In [8]:
# 清洗数据
def clean(content):
    #将数据中的emoji表情转化为文字
    content = emoji.demojize(content)
    #过滤其中的html标签
    content = re.sub('<.*>','',content)
    return content

In [9]:
# 对每个问题应用上一clean函数进行清洗
QApares_df['question_after_preprocessing'] = QApares_df['question_before_preprocessing'].apply(clean)

In [11]:
# 按照一般的流程，之后应该是对问题进行分词，但由于我们的数据已经是分好词的数据，所以这里不再进行中文分词，而是直接将分好的词以列表的形式存储。
QApares_df['question_after_preprocessing'] = QApares_df['question_after_preprocessing'].apply(lambda x:x.split())

In [12]:
# 对question_after_preprocessing列去除单词中的空格回车等符号
def strip(wordList):
    return [word.strip() for word in wordList if word.strip()!='']
QApares_df['question_after_preprocessing'] = QApares_df['question_after_preprocessing'].apply(strip)

In [13]:
# 读取data/stopWord.json中保存的的停用词表，并保存在列表中
# 中文停用词表的下载地址:https://github.com/goto456/stopwords/
with open("data/stopWord.json","r") as f:
    stopWords = f.read().split("\n")

In [14]:
# 去除停用词
def rm_stop_word(wordList):
    return [word for word in wordList if word not in stopWords]
QApares_df['question_after_preprocessing'] = QApares_df['question_after_preprocessing'].apply(rm_stop_word)

In [15]:
# 去除低频词
allWords = [word for question in QApares_df['question_after_preprocessing'] for word in question] #所有词组成的列表
freWord = Counter(allWords) #统计词频，一个字典，键为词，值为词出现的次数
highFreWords = [word for word in freWord.keys() if freWord[word]>5] # 词频超过5的词列表, 剩下的词去掉
def rm_low_fre_word(content):
    return [word for word in content if word in highFreWords]

# 去除低频词
QApares_df['question_after_preprocessing'] = QApares_df['question_after_preprocessing'].apply(rm_low_fre_word)

In [17]:
# 只保留question,answer和question_after_preprocessing列
QApares_df = QApares_df[['question','answer','question_after_preprocessing']]
#查看预处理之后的数据
QApares_df.head()

,question,answer,question_after_preprocessing
0,买二份有没有少点呀,亲亲真的不好意思我们已经是优惠价了呢小本生意请亲谅解,"[买, 二份, 有没有, 少点]"
1,那就等你们处理喽,好的亲退了,[处理]
2,那我不喜欢,颜色的话一般茶刀茶针和二合一的话都是红木檀和黑木檀哦,[喜欢]
3,不是免运费,本店茶具订单满99包邮除宁夏青海内蒙古海南新疆西藏满39包邮,"[免, 运费]"
4,好吃吗,好吃的,[好吃]


In [18]:
#由于保存为csv格式会将数据默认保存为str格式，但我们的数据中question_after_preprocessing列中每一项为列表形式，保存为csv格式会增加我们后面处理数据的难度，所以这里我们将保存为pickle形式
import pickle
with open('data/question_answer_pares.pkl','wb') as f:
    pickle.dump(QApares_df,f)